# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [166]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import re
import nltk
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix

nltk.download('punkt')
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [208]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', con=engine)
X = df['message'].values
y = df.drop(['id', 'message', 'original', 'genre' ], axis=1).values

In [210]:
np.unique(y)

array([0, 1, 2])

In [211]:
from statistics import mode

for i, x in enumerate(y):
    for j, a in enumerate(x):
        if x[j] == 2:
            y[i][j] = mode(x)

In [212]:
np.unique(y)

array([0, 1])

### 2. Write a tokenization function to process your text data

In [194]:
def tokenize(text):
    
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    tokens = word_tokenize(text)
    
    clean_tokens = [lemmatizer.lemmatize(word).strip().lower() for word in tokens if word not in stop_words]
    return clean_tokens

In [195]:
tokenize(X[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [196]:
# Build the ML pipeline
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()
forest = RandomForestClassifier(random_state=1)
clf = MultiOutputClassifier(forest, n_jobs=1)

pipeline = Pipeline([('vect', vect), ('tfidf', tfidf), ('clf', clf)])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [197]:
# split the data to train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...1,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [198]:
y_pred = pipeline.predict(X_test)

In [199]:
pipeline.score(X, y)

0.67638083613060729

In [202]:
labels = np.unique(y_pred)
#confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
print(labels)

[0 1]


In [203]:
(y_pred.shape, y_test.shape)

((8652, 36), (8652, 36))

In [215]:
(y_pred.dtype, y_test.dtype)

(dtype('int64'), dtype('int64'))

In [207]:
(type(y_test), type(y_pred))

(numpy.ndarray, numpy.ndarray)

In [213]:
from sklearn.utils.multiclass import type_of_target

(type_of_target(y_test), type_of_target(y_pred))

('multilabel-indicator', 'multilabel-indicator')

In [183]:
classification_report(y_pred, y_test)

ValueError: Mix type of y not allowed, got types {'multiclass-multioutput', 'multilabel-indicator'}

In [184]:
accuracy = ((y_pred == y_test).mean().mean())
print(accuracy)

0.943783068783


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.